### Revisão de probabilidade

Primeiramente, sabemos que:
$$P(X) = \frac{total_{x}}{total_{eventos}}$$

Isto é, a probabilidade de um evento X ocorrer, é igual ao total de vezes que ele ocorre em uma amostragem sob o total da amostragem.

Também sabemos a probabilidade que a probabilidade de dois eventos ocorrerem simultaneamente é dado por:

$$P(A,B) = P(A\cap B) = \frac{|A \cap B|}{|A \cup B|}$$

Por último é sabido que a probabilidade de A dado B é:

$$P(A | B) = \frac{P(A \cap B)}{P(B)}$$

Com isso, sem perda de generalidade, deduzimos que:

$$P(A | B) = \frac{P(B | A) * P(A)}{P(B)}$$

A essa equação que definimos como regra de Bayes.

### Laboratório 2

#### Naive Bayes

Faz uso da regra de Bayes para computar a probabilidade de um evento. É chamada de "naive" porque presupõe que as variáveis são independentes.

A regra de Bayes nesse método é aplicado da seguinte forma

$$\prod_{i=1}^{m}{\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)}}$$

#### Desvatagens do método

    1. Assume que as palavras são indepentes umas das outras. Mas sabemos que o contexto influencia no texto.
    2. Depende da frequência do corpo do texto. Características importantes podem passar, caso haja falta de frequência no texto.

In [2]:
#Mesmo início de código da outra aula
import numpy as np
import pandas as pd

corpo = [
    "Eu gosto de comer biscoito",
    "Eu odeio você",
    "Eu te amo!!",
    "Você é o amor da minha vida",
    "Vá para a baixa da égua",
    "Você é uma pessoa repugnante",
    "Você é desagradável",
    "Você é amável"
]

classes = [
    1,
    0,
    1,
    1,
    0,
    0,
    0,
    1
]

dados = pd.DataFrame({"corpo":corpo, "classes":classes})
vocabulario = {}
indices = {}
pontuacao = ['!',","]
palavrasPausa = ["a", "é", "uma", "de", "o", "da"]
def criarVocabulario(corpo):
    v = {}
    i = {}
    index = 0
    for frase in corpo:
        for palavra in frase.split(" "):
            for p in pontuacao:
                palavra = palavra.replace(p,'')
            if palavra not in list(v.keys()) and palavra not in palavrasPausa:
                v[palavra] = index
                i[index] = palavra
                index+=1
    return v,i    

vocabulario, indices = criarVocabulario(corpo)

def medirFrequencia(frase):
    array = np.zeros(len(list(vocabulario.keys())))
    for palavra in frase.split(" "):
        for p in pontuacao:
            palavra = palavra.replace(p,'')
        if palavra not in palavrasPausa:
            array[vocabulario[palavra]] += 1
    return array
def somarFrequencia(data):
    arrayPositivo = np.zeros(len(list(vocabulario.keys())))
    arrayNegativo = np.zeros(len(list(vocabulario.keys())))
    for i in range(0,data.shape[0]):
        if data.iloc[i,1] == 1:
            arrayPositivo += medirFrequencia(data.iloc[i,0])
        else:
            arrayNegativo += medirFrequencia(data.iloc[i,0])
    return arrayPositivo, arrayNegativo

positivos, negativos = somarFrequencia(dados)

In [3]:
positivos = positivos/np.sum(positivos)
negativos = negativos/np.sum(negativos)
positivos,negativos

(array([0.15384615, 0.07692308, 0.07692308, 0.07692308, 0.        ,
        0.        , 0.07692308, 0.07692308, 0.15384615, 0.07692308,
        0.07692308, 0.07692308, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.07692308]),
 array([0.08333333, 0.        , 0.        , 0.        , 0.08333333,
        0.08333333, 0.        , 0.        , 0.16666667, 0.        ,
        0.        , 0.        , 0.08333333, 0.08333333, 0.08333333,
        0.08333333, 0.08333333, 0.08333333, 0.08333333, 0.        ]))

In [4]:
def naiveBayes(frase, positivos,negativos):
    prob = 1
    for palavra in frase.split(" "):
        for p in pontuacao:
            palavra = palavra.replace(p,'')
        if(palavra in palavrasPausa or negativos[vocabulario[palavra]] == 0):
            continue
        prob *= positivos[vocabulario[palavra]]/negativos[vocabulario[palavra]]
    return prob > 1

def runNaiveBayes(data):
    result = []
    positivos, negativos = somarFrequencia(dados)
    positivos = positivos/np.sum(positivos)
    negativos = negativos/np.sum(negativos)
    for frase in data.corpo:
        result.append(naiveBayes(frase,positivos,negativos))
    return result

print("Acc",np.sum(np.array(runNaiveBayes(dados)) == np.array(dados.classes))/len(classes))

Acc 0.75


### Laplacian Smoothing

Até então a tivemos o cálculo dos termos da seguinte forma

$$P(w_{i}|class) = \frac{freq(w_{i},class)}{N_{class}}$$

Mas obtivemos a problematica de que quando calculamos $\prod_{i=1}^{m}{\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)}}$, poderiamos ter divisão por zero se uma palavra $w_{i}$ não existisse em uma classe. Contornamos isso dando "continue" quando o denominador era zero.

No caso, esse método serve justamente para que não seja feito esse contorno. Ele basicamente adiciona 1 no numerador, e para garantir a normalização dos valores e que a soma das probabilidades ainda resulte em 1, ele soma $V$ ao denominador, onde $V$ é o total de palavras únicas do nosso vocabulário. O resultado fica:

$$P(w_{i}|class) = \frac{freq(w_{i},class) + 1}{N_{class} + V}$$

In [12]:
def runNaiveBayesNormalized(data):
    result = []
    positivos, negativos = somarFrequencia(dados)
    positivos = (positivos + 1)/(np.sum(positivos) +len(vocabulario))
    negativos = (negativos + 1)/(np.sum(negativos) +len(vocabulario))
    for frase in data.corpo:
        result.append(naiveBayes(frase,positivos,negativos))
    return result

print("Acc",np.sum(np.array(runNaiveBayesNormalized(dados)) == np.array(dados.classes))/len(classes))

Acc 1.0


### Resultado

Interessante que a própria precisão aumentou ao aplicar a tecnica de normalização. Em suma, esse estudo foi sobre o algoritmo Naive Bayes. 

### Naive Bayes com Log Likelihood

Aprendemos que o método de Naive Bayes se baseia no seguinte produtório:

$$\prod_{i=1}^{m}{\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)}}$$

Porém sabemos que trabalhar com produtos e probabilidade não é algo interessante, visto que uma série de produtos flutuantes pode gerar um underflow, perdendo a precisão das casas decimais. Com isso propõe-se o seguinte método. Se para que tenhamos uma classe positiva segue que:

$$\prod_{i=1}^{m}{\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)}} > 1$$

Então, se aplicarmos log teremos

$$log(\prod_{i=1}^{m}{\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)}}) > log(1)$$

E por tanto:

$$\sum_{i=1}^{m}{log(\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)})} > 0$$

com isso evitamos trabalhar com produtório e resolvemos o problema de underflow.


#### Mas e se os dados estiverem desbalanceados ? 

Dados desbalanceados implicam que as palavras neutras estão mal divididas tendendo a estarem em conjuntos positivos ou negativos. Isso, a priore causa um problema no cálculo da naive bayes. Por isso se propõe o log prior.

O log prior é um fator que é acrescido na fórmula da log likelihood. Ele é expresso pela fórmula.

$$\frac{P(Pos)}{P(Neg)}$$

Com isso, a log likelihood ajustada fica na forma:

$$\frac{P(Pos)}{P(Neg)} \sum_{i=1}^{m}{log(\frac{P(w_{i}|Pos)}{P(w_{i}|Neg)})}$$

Como o log prior deve ser igual a um (caso os dados estejam balanceados), não afetará o cálculo. Todavia, se for diferente de 1, vai servir de peso para deixar as probabilidades mais justas.


In [20]:
def naiveBayesLog(frase, positivos,negativos):
    prob = 0
    for palavra in frase.split(" "):
        for p in pontuacao:
            palavra = palavra.replace(p,'')
        if palavra not in list(vocabulario.keys()):
            continue
        if(palavra in palavrasPausa or negativos[vocabulario[palavra]] == 0):
            continue
        logPrior = np.sum(positivos)/np.sum(negativos)
        prob += np.log(positivos[vocabulario[palavra]]/negativos[vocabulario[palavra]])
    return (logPrior * prob)

def runNaiveBayesLogNormalized(data):
    result = []
    positivos, negativos = somarFrequencia(dados)
    positivos = (positivos + 1)/(np.sum(positivos) +len(vocabulario))
    negativos = (negativos + 1)/(np.sum(negativos) +len(vocabulario))
    for frase in data.corpo:
        result.append(naiveBayesLog(frase,positivos,negativos) > 0)
    return result

print("Acc",np.sum(np.array(runNaiveBayesLogNormalized(dados)) == np.array(dados.classes))/len(classes))

Acc 1.0


#### Aplicações do método de Naive Bayes

Author identification

Spam filtering 

Information retrieval 

Word disambiguation 

This method is usually used as a simple baseline. It also really fast.